<a href="https://colab.research.google.com/github/duy7590/portfolio_tracker/blob/master/Investment_Portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**References:**  

https://www.datacamp.com/community/tutorials/finance-python-trading

https://github.com/datacamp/datacamp-community-tutorials/blob/master/Python%20Finance%20Tutorial%20For%20Beginners/Python%20For%20Finance%20Beginners%20Tutorial.ipynb

https://pypi.python.org/pypi/fix-yahoo-finance

http://www.learndatasci.com/python-finance-part-yahoo-finance-api-pandas-matplotlib/

In [1]:
# Import initial libraries

import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Imports in order to be able to use Plotly offline.
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import plotly.graph_objs as go

print(__version__) # requires version >= 1.9.0

init_notebook_mode(connected=True)

3.6.1


In [3]:
# Import the Sample worksheet with acquisition dates and initial cost basis:

import pandas as pd
import io
import requests
url="https://raw.githubusercontent.com/duy7590/portfolio_tracker/master/data.csv"
s=requests.get(url).content
portfolio_df = pd.read_csv(io.StringIO(s.decode('utf-8')))
portfolio_df['Acquisition Date'] = portfolio_df['Acquisition Date'].astype('datetime64[ns]') 
portfolio_df.head(10)

,Acquisition Date,Ticker,Quantity,Unit Cost,Cost Basis,Start of Year
0,2013-02-07,AAPL,125,65.40,"8,175.00",12/29/17
1,2014-02-27,JNJ,100,81.90,"8,190.00",12/29/17
2,2013-06-04,MCD,100,84.99,"8,499.00",12/29/17
3,2015-12-14,MTCH,600,13.63,"8,178.00",12/29/17
4,2016-01-14,NFLX,75,108.71,"8,153.25",12/29/17
5,2013-08-14,WMT,125,68.30,"8,537.50",12/29/17
6,2013-12-13,FB,150,53.32,"7,998.00",12/29/17
7,2015-01-05,TWTR,225,36.38,"8,185.50",12/29/17


In [4]:
# Confirm that you have 8 values for each column.

portfolio_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 6 columns):
Acquisition Date    8 non-null datetime64[ns]
Ticker              8 non-null object
Quantity            8 non-null int64
Unit Cost           8 non-null float64
Cost Basis          8 non-null object
Start of Year       8 non-null object
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 456.0+ bytes


In [5]:
# Date Ranges for SP 500 and for all tickers
# Modify these date ranges each week.

# The below will pull back stock prices from 2010 until end date specified.
start_sp = datetime.datetime(2013, 1, 1)
end_sp = datetime.datetime(2018, 3, 9)

# This variable is used for YTD performance.
end_of_last_year = datetime.datetime(2017, 12, 29)

# These are separate if for some reason want different date range than SP.
stocks_start = datetime.datetime(2013, 1, 1)
stocks_end = datetime.datetime(2020, 6, 6)

In [6]:
# Leveraged from the helpful Datacamp Python Finance trading blog post.

from pandas_datareader import data as pdr
import fix_yahoo_finance as yf
yf.pdr_override() # <== that's all it takes :-)

sp500 = pdr.get_data_yahoo('^GSPC', 
                           start_sp,
                             end_sp)
                          
sp500.head()

[*********************100%***********************]  1 of 1 downloaded


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2013-01-02,1426.189941,1462.430054,1426.189941,1462.420044,1462.420044,4202600000
2013-01-03,1462.420044,1465.469971,1455.530029,1459.369995,1459.369995,3829730000
2013-01-04,1459.369995,1467.939941,1458.989990,1466.469971,1466.469971,3424290000
2013-01-07,1466.469971,1466.469971,1456.619995,1461.890015,1461.890015,3304970000
2013-01-08,1461.890015,1461.890015,1451.640015,1457.150024,1457.150024,3601600000


In [7]:
sp500.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-03-02,2658.889893,2696.250000,2647.320068,2691.250000,2691.250000,3882450000
2018-03-05,2681.060059,2728.090088,2675.750000,2720.939941,2720.939941,3710810000
2018-03-06,2730.179932,2732.080078,2711.260010,2728.120117,2728.120117,3370690000
2018-03-07,2710.179932,2730.600098,2701.739990,2726.800049,2726.800049,3393270000
2018-03-08,2732.750000,2740.449951,2722.649902,2738.969971,2738.969971,3212320000


In [8]:
# Create a dataframe with only the Adj Close column as that's all we need for this analysis.

sp_500_adj_close = sp500[['Adj Close']].reset_index()

In [9]:
sp_500_adj_close.tail()

,Date,Adj Close
1300,2018-03-02,2691.250000
1301,2018-03-05,2720.939941
1302,2018-03-06,2728.120117
1303,2018-03-07,2726.800049
1304,2018-03-08,2738.969971


In [10]:
# Adj Close for the EOY in 2017 in order to run comparisons versus stocks YTD performances.

sp_500_adj_close_start = sp_500_adj_close[sp_500_adj_close['Date']==end_of_last_year]
sp_500_adj_close_start

,Date,Adj Close
1258,2017-12-29,2673.610107


In [11]:
# Generate a dynamic list of tickers to pull from Yahoo Finance API based on the imported file with tickers.
tickers = portfolio_df['Ticker'].unique()
tickers

array(['AAPL', 'JNJ', 'MCD', 'MTCH', 'NFLX', 'WMT', 'FB', 'TWTR'],
      dtype=object)

In [12]:
# Stock comparison code

def get(tickers, startdate, enddate):
    def data(ticker):
        return (pdr.get_data_yahoo(ticker, start=startdate, end=enddate))
    datas = map(data, tickers)
    return(pd.concat(datas, keys=tickers, names=['Ticker', 'Date']))
               
all_data = get(tickers, stocks_start, stocks_end)

[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded


In [13]:
all_data.head()

Open       High  ...  Adj Close     Volume
Ticker Date                              ...                      
AAPL   2013-01-02  79.117142  79.285713  ...  68.502106  140129500
       2013-01-03  78.268570  78.524284  ...  67.637436   88241300
       2013-01-04  76.709999  76.947144  ...  65.753418  148583400
       2013-01-07  74.571426  75.614288  ...  65.366646  121039100
       2013-01-08  75.601425  75.984283  ...  65.542587  114676800

[5 rows x 6 columns]

In [14]:
# Also only pulling the ticker, date and adj. close columns for our tickers.

adj_close = all_data[['Adj Close']].reset_index()
adj_close.head()

,Ticker,Date,Adj Close
0,AAPL,2013-01-02,68.502106
1,AAPL,2013-01-03,67.637436
2,AAPL,2013-01-04,65.753418
3,AAPL,2013-01-07,65.366646
4,AAPL,2013-01-08,65.542587


In [15]:
# Grabbing the ticker close from the end of last year
adj_close_start = adj_close[adj_close['Date']==end_of_last_year]
adj_close_start.head()

,Ticker,Date,Adj Close
1258,AAPL,2017-12-29,163.427597
3128,JNJ,2017-12-29,130.557571
4998,MCD,2017-12-29,161.814545
6141,MTCH,2017-12-29,29.799627
8011,NFLX,2017-12-29,191.960007


In [16]:
# Grab the latest stock close price

adj_close_latest = adj_close[adj_close['Date']==(stocks_end-datetime.timedelta(1))]
adj_close_latest

,Ticker,Date,Adj Close
1869,AAPL,2020-06-05,331.500000
3739,JNJ,2020-06-05,147.300003
5609,MCD,2020-06-05,197.160004
6752,MTCH,2020-06-05,90.089996
8622,NFLX,2020-06-05,419.600006
10492,WMT,2020-06-05,121.559998
12362,FB,2020-06-05,230.770004
14017,TWTR,2020-06-05,34.869999


In [17]:
adj_close_latest.set_index('Ticker', inplace=True)
adj_close_latest.head()

,Date,Adj Close
Ticker,,
AAPL,2020-06-05,331.500000
JNJ,2020-06-05,147.300003
MCD,2020-06-05,197.160004
MTCH,2020-06-05,90.089996
NFLX,2020-06-05,419.600006


In [18]:
portfolio_df.set_index(['Ticker'], inplace=True)

portfolio_df.head()

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year
Ticker,,,,,
AAPL,2013-02-07,125,65.40,"8,175.00",12/29/17
JNJ,2014-02-27,100,81.90,"8,190.00",12/29/17
MCD,2013-06-04,100,84.99,"8,499.00",12/29/17
MTCH,2015-12-14,600,13.63,"8,178.00",12/29/17
NFLX,2016-01-14,75,108.71,"8,153.25",12/29/17


In [19]:
# Merge the portfolio dataframe with the adj close dataframe; they are being joined by their indexes.

merged_portfolio = pd.merge(portfolio_df, adj_close_latest, left_index=True, right_index=True)
merged_portfolio

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Date,Adj Close
Ticker,,,,,,,
AAPL,2013-02-07,125,65.40,"8,175.00",12/29/17,2020-06-05,331.500000
JNJ,2014-02-27,100,81.90,"8,190.00",12/29/17,2020-06-05,147.300003
MCD,2013-06-04,100,84.99,"8,499.00",12/29/17,2020-06-05,197.160004
MTCH,2015-12-14,600,13.63,"8,178.00",12/29/17,2020-06-05,90.089996
NFLX,2016-01-14,75,108.71,"8,153.25",12/29/17,2020-06-05,419.600006
WMT,2013-08-14,125,68.30,"8,537.50",12/29/17,2020-06-05,121.559998
FB,2013-12-13,150,53.32,"7,998.00",12/29/17,2020-06-05,230.770004
TWTR,2015-01-05,225,36.38,"8,185.50",12/29/17,2020-06-05,34.869999


In [20]:
# The below creates a new column which is the ticker return; takes the latest adjusted close for each position
# and divides that by the initial share cost.

merged_portfolio['ticker return'] = merged_portfolio['Adj Close'] / merged_portfolio['Unit Cost'] - 1

merged_portfolio

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Date,Adj Close,ticker return
Ticker,,,,,,,,
AAPL,2013-02-07,125,65.40,"8,175.00",12/29/17,2020-06-05,331.500000,4.068807
JNJ,2014-02-27,100,81.90,"8,190.00",12/29/17,2020-06-05,147.300003,0.798535
MCD,2013-06-04,100,84.99,"8,499.00",12/29/17,2020-06-05,197.160004,1.319802
MTCH,2015-12-14,600,13.63,"8,178.00",12/29/17,2020-06-05,90.089996,5.609684
NFLX,2016-01-14,75,108.71,"8,153.25",12/29/17,2020-06-05,419.600006,2.859811
WMT,2013-08-14,125,68.30,"8,537.50",12/29/17,2020-06-05,121.559998,0.779795
FB,2013-12-13,150,53.32,"7,998.00",12/29/17,2020-06-05,230.770004,3.328020
TWTR,2015-01-05,225,36.38,"8,185.50",12/29/17,2020-06-05,34.869999,-0.041506


In [21]:
merged_portfolio.reset_index(inplace=True)

In [22]:
type(merged_portfolio['Acquisition Date'][1])

pandas._libs.tslibs.timestamps.Timestamp

In [23]:
# Here we are merging the new dataframe with the sp500 adjusted closes since the sp start price based on 
# each ticker's acquisition date and sp500 close date.

merged_portfolio_sp = pd.merge(merged_portfolio, sp_500_adj_close, left_on='Acquisition Date', right_on='Date')
# .set_index('Ticker')

merged_portfolio_sp.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Date_x,Adj Close_x,ticker return,Date_y,Adj Close_y
0,AAPL,2013-02-07,125,65.40,"8,175.00",12/29/17,2020-06-05,331.500000,4.068807,2013-02-07,1509.390015
1,JNJ,2014-02-27,100,81.90,"8,190.00",12/29/17,2020-06-05,147.300003,0.798535,2014-02-27,1854.290039
2,MCD,2013-06-04,100,84.99,"8,499.00",12/29/17,2020-06-05,197.160004,1.319802,2013-06-04,1631.380005
3,MTCH,2015-12-14,600,13.63,"8,178.00",12/29/17,2020-06-05,90.089996,5.609684,2015-12-14,2021.939941
4,NFLX,2016-01-14,75,108.71,"8,153.25",12/29/17,2020-06-05,419.600006,2.859811,2016-01-14,1921.839966


In [24]:
# We will delete the additional date column which is created from this merge.
# We then rename columns to Latest Date and then reflect Ticker Adj Close and SP 500 Initial Close.

del merged_portfolio_sp['Date_y']

merged_portfolio_sp.rename(columns={'Date_x': 'Latest Date', 'Adj Close_x': 'Ticker Adj Close'
                                    , 'Adj Close_y': 'SP 500 Initial Close'}, inplace=True)

merged_portfolio_sp.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close
0,AAPL,2013-02-07,125,65.40,"8,175.00",12/29/17,2020-06-05,331.500000,4.068807,1509.390015
1,JNJ,2014-02-27,100,81.90,"8,190.00",12/29/17,2020-06-05,147.300003,0.798535,1854.290039
2,MCD,2013-06-04,100,84.99,"8,499.00",12/29/17,2020-06-05,197.160004,1.319802,1631.380005
3,MTCH,2015-12-14,600,13.63,"8,178.00",12/29/17,2020-06-05,90.089996,5.609684,2021.939941
4,NFLX,2016-01-14,75,108.71,"8,153.25",12/29/17,2020-06-05,419.600006,2.859811,1921.839966


In [25]:
# This new column determines what SP 500 equivalent purchase would have been at purchase date of stock.
merged_portfolio_sp['Equiv SP Shares'] = merged_portfolio_sp['Cost Basis'] / merged_portfolio_sp['SP 500 Initial Close']
merged_portfolio_sp.head()

TypeError: ignored

In [ ]:
# We are joining the developing dataframe with the sp500 closes again, this time with the latest close for SP.
merged_portfolio_sp_latest = pd.merge(merged_portfolio_sp, sp_500_adj_close, left_on='Latest Date', right_on='Date')

merged_portfolio_sp_latest.head()

In [ ]:
# Once again need to delete the new Date column added as it's redundant to Latest Date.  
# Modify Adj Close from the sp dataframe to distinguish it by calling it the SP 500 Latest Close.

del merged_portfolio_sp_latest['Date']

merged_portfolio_sp_latest.rename(columns={'Adj Close': 'SP 500 Latest Close'}, inplace=True)

merged_portfolio_sp_latest.head()

In [ ]:
# Percent return of SP from acquisition date of position through latest trading day.
merged_portfolio_sp_latest['SP Return'] = merged_portfolio_sp_latest['SP 500 Latest Close'] / merged_portfolio_sp_latest['SP 500 Initial Close'] - 1

# This is a new column which takes the tickers return and subtracts the sp 500 equivalent range return.
merged_portfolio_sp_latest['Abs. Return Compare'] = merged_portfolio_sp_latest['ticker return'] - merged_portfolio_sp_latest['SP Return']

# This is a new column where we calculate the ticker's share value by multiplying the original quantity by the latest close.
merged_portfolio_sp_latest['Ticker Share Value'] = merged_portfolio_sp_latest['Quantity'] * merged_portfolio_sp_latest['Ticker Adj Close']

# We calculate the equivalent SP 500 Value if we take the original SP shares * the latest SP 500 share price.
merged_portfolio_sp_latest['SP 500 Value'] = merged_portfolio_sp_latest['Equiv SP Shares'] * merged_portfolio_sp_latest['SP 500 Latest Close']

# This is a new column where we take the current market value for the shares and subtract the SP 500 value.
merged_portfolio_sp_latest['Abs Value Compare'] = merged_portfolio_sp_latest['Ticker Share Value'] - merged_portfolio_sp_latest['SP 500 Value']

# This column calculates profit / loss for stock position.
merged_portfolio_sp_latest['Stock Gain / (Loss)'] = merged_portfolio_sp_latest['Ticker Share Value'] - merged_portfolio_sp_latest['Cost Basis']

# This column calculates profit / loss for SP 500.
merged_portfolio_sp_latest['SP 500 Gain / (Loss)'] = merged_portfolio_sp_latest['SP 500 Value'] - merged_portfolio_sp_latest['Cost Basis']

merged_portfolio_sp_latest.head()

In [ ]:
# Merge the overall dataframe with the adj close start of year dataframe for YTD tracking of tickers.
# Should not need to do the outer join;

merged_portfolio_sp_latest_YTD = pd.merge(merged_portfolio_sp_latest, adj_close_start, on='Ticker')
# , how='outer'

merged_portfolio_sp_latest_YTD.head()

In [ ]:
# Deleting date again as it's an unnecessary column.  Explaining that new column is the Ticker Start of Year Close.

del merged_portfolio_sp_latest_YTD['Date']

merged_portfolio_sp_latest_YTD.rename(columns={'Adj Close': 'Ticker Start Year Close'}, inplace=True)

merged_portfolio_sp_latest_YTD.head()

In [ ]:
# Join the SP 500 start of year with current dataframe for SP 500 ytd comparisons to tickers.

merged_portfolio_sp_latest_YTD_sp = pd.merge(merged_portfolio_sp_latest_YTD, sp_500_adj_close_start
                                             , left_on='Start of Year', right_on='Date')

merged_portfolio_sp_latest_YTD_sp.head()

In [ ]:
# Deleting another unneeded Date column.

del merged_portfolio_sp_latest_YTD_sp['Date']

# Renaming so that it's clear this column is SP 500 start of year close.
merged_portfolio_sp_latest_YTD_sp.rename(columns={'Adj Close': 'SP Start Year Close'}, inplace=True)

# YTD return for portfolio position.
merged_portfolio_sp_latest_YTD_sp['Share YTD'] = merged_portfolio_sp_latest_YTD_sp['Ticker Adj Close'] / merged_portfolio_sp_latest_YTD_sp['Ticker Start Year Close'] - 1

# YTD return for SP to run compares.
merged_portfolio_sp_latest_YTD_sp['SP 500 YTD'] = merged_portfolio_sp_latest_YTD_sp['SP 500 Latest Close'] / merged_portfolio_sp_latest_YTD_sp['SP Start Year Close'] - 1

merged_portfolio_sp_latest_YTD_sp.head()

In [ ]:
merged_portfolio_sp_latest_YTD_sp = merged_portfolio_sp_latest_YTD_sp.sort_values(by='Ticker', ascending=True)
merged_portfolio_sp_latest_YTD_sp

In [ ]:
# Cumulative sum of original investment
merged_portfolio_sp_latest_YTD_sp['Cum Invst'] = merged_portfolio_sp_latest_YTD_sp['Cost Basis'].cumsum()

# Cumulative sum of Ticker Share Value (latest FMV based on initial quantity purchased).
merged_portfolio_sp_latest_YTD_sp['Cum Ticker Returns'] = merged_portfolio_sp_latest_YTD_sp['Ticker Share Value'].cumsum()

# Cumulative sum of SP Share Value (latest FMV driven off of initial SP equiv purchase).
merged_portfolio_sp_latest_YTD_sp['Cum SP Returns'] = merged_portfolio_sp_latest_YTD_sp['SP 500 Value'].cumsum()

# Cumulative CoC multiple return for stock investments
merged_portfolio_sp_latest_YTD_sp['Cum Ticker ROI Mult'] = merged_portfolio_sp_latest_YTD_sp['Cum Ticker Returns'] / merged_portfolio_sp_latest_YTD_sp['Cum Invst']

merged_portfolio_sp_latest_YTD_sp.head()


## Assessing Where Positions are At versus Highest Close

In [ ]:
# Referencing the adj_close dataframe from above

adj_close.head()

In [ ]:
portfolio_df.head()

In [ ]:
# Need to factor in that some positions were purchased much more recently than others.
# Join adj_close dataframe with portfolio in order to have acquisition date.

portfolio_df.reset_index(inplace=True)

adj_close_acq_date = pd.merge(adj_close, portfolio_df, on='Ticker')

adj_close_acq_date.head()

In [ ]:
# delete_columns = ['Quantity', 'Unit Cost', 'Cost Basis', 'Start of Year']

del adj_close_acq_date['Quantity']
del adj_close_acq_date['Unit Cost']
del adj_close_acq_date['Cost Basis']
del adj_close_acq_date['Start of Year']

# Sort by these columns in this order in order to make it clearer where compare for each position should begin.
adj_close_acq_date.sort_values(by=['Ticker', 'Acquisition Date', 'Date'], ascending=[True, True, True], inplace=True)

In [ ]:
# Anything less than 0 means that the stock close was prior to acquisition.

adj_close_acq_date['Date Delta'] = pd.to_datetime(adj_close_acq_date['Date']) - pd.to_datetime(adj_close_acq_date['Acquisition Date'])

adj_close_acq_date['Date Delta'] = adj_close_acq_date[['Date Delta']].apply(pd.to_numeric)  

adj_close_acq_date.head()




In [ ]:
# Modified the dataframe being evaluated to look at highest close which occurred after Acquisition Date (aka, not prior to purchase).

adj_close_acq_date_modified = adj_close_acq_date[adj_close_acq_date['Date Delta']>=0]

adj_close_acq_date_modified.head()

In [ ]:
# This pivot table will index on the Ticker and Acquisition Date, and find the max adjusted close.

adj_close_pivot = adj_close_acq_date_modified.pivot_table(index=['Ticker', 'Acquisition Date'], values='Adj Close', aggfunc=np.max)

adj_close_pivot.reset_index(inplace=True)

adj_close_pivot

In [ ]:
# Merge the adj close pivot table with the adj_close table in order to grab the date of the Adj Close High (good to know).

adj_close_pivot_merged = pd.merge(adj_close_pivot, adj_close
                                             , on=['Ticker', 'Adj Close'])

adj_close_pivot_merged.head()

In [ ]:
# Merge the Adj Close pivot table with the master dataframe to have the closing high since you have owned the stock.

merged_portfolio_sp_latest_YTD_sp_closing_high = pd.merge(merged_portfolio_sp_latest_YTD_sp, adj_close_pivot_merged
                                             , on=['Ticker', 'Acquisition Date'])

# Renaming so that it's clear that the new columns are two year closing high and two year closing high date.
merged_portfolio_sp_latest_YTD_sp_closing_high.rename(columns={'Adj Close': 'Closing High Adj Close', 'Date': 'Closing High Adj Close Date'}, inplace=True)

merged_portfolio_sp_latest_YTD_sp_closing_high['Pct off High'] = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker Adj Close'] / merged_portfolio_sp_latest_YTD_sp_closing_high['Closing High Adj Close'] - 1 

merged_portfolio_sp_latest_YTD_sp_closing_high

In [ ]:
# Not needed for this blog post -- this is if you have multiple positions for the same ticker with different acquisition dates.
# merged_portfolio_sp_latest_YTD_sp_closing_high['Counts'] = merged_portfolio_sp_latest_YTD_sp_closing_high.index

# merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker #'] = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'].map(str) + ' ' + merged_portfolio_sp_latest_YTD_sp_closing_high['Counts'].map(str)

# merged_portfolio_sp_latest_YTD_sp_closing_high.head()

## YTD and Trailing Stop Charts

In [ ]:
# Ploty is an outstanding resource for interactive charts.

trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp['Share YTD'][0:10],
    name = 'Ticker YTD')

trace2 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp['SP 500 YTD'][0:10],
    name = 'SP500 YTD')
    
data = [trace1, trace2]

layout = go.Layout(title = 'YTD Return vs S&P 500 YTD'
    , barmode = 'group'
    , yaxis=dict(title='Returns', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.8,y=1)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
# Current Share Price versus Closing High Since Purchased

trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Pct off High'][0:10],
    name = 'Pct off High')
    
data = [trace1]

layout = go.Layout(title = 'Adj Close % off of High'
    , barmode = 'group'
    , yaxis=dict(title='% Below Adj Close High', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.8,y=1)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

## Total Return Comparison Charts

In [ ]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['ticker return'][0:10],
    name = 'Ticker Total Return')

trace2 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['SP Return'][0:10],
    name = 'SP500 Total Return')
    
data = [trace1, trace2]

layout = go.Layout(title = 'Total Return vs S&P 500'
    , barmode = 'group'
    , yaxis=dict(title='Returns', tickformat=".2%")
    , xaxis=dict(title='Ticker', tickformat=".2%")
    , legend=dict(x=.8,y=1)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

## Cumulative Returns Over Time

In [ ]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Stock Gain / (Loss)'][0:10],
    name = 'Ticker Total Return ($)')

trace2 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['SP 500 Gain / (Loss)'][0:10],
    name = 'SP 500 Total Return ($)')

trace3 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['ticker return'][0:10],
    name = 'Ticker Total Return %',
    yaxis='y2')

data = [trace1, trace2, trace3]

layout = go.Layout(title = 'Gain / (Loss) Total Return vs S&P 500'
    , barmode = 'group'
    , yaxis=dict(title='Gain / (Loss) ($)')
    , yaxis2=dict(title='Ticker Return', overlaying='y', side='right', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.75,y=1)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum Invst'],
    # mode = 'lines+markers',
    name = 'Cum Invst')

trace2 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum SP Returns'],
    # mode = 'lines+markers',
    name = 'Cum SP500 Returns')

trace3 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum Ticker Returns'],
    # mode = 'lines+markers',
    name = 'Cum Ticker Returns')

trace4 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum Ticker ROI Mult'],
    # mode = 'lines+markers',
    name = 'Cum ROI Mult'
    , yaxis='y2')


data = [trace1, trace2, trace3, trace4]

layout = go.Layout(title = 'Total Cumulative Investments Over Time'
    , barmode = 'group'
    , yaxis=dict(title='Returns')
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.4,y=1)
    , yaxis2=dict(title='Cum ROI Mult', overlaying='y', side='right')               
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)